# RAGAS Evaluation for Bedrock RAG Solutions
Automated evaluation of RAG pipeline performance using RAGAS metrics

**RAGAS Metrics:**
- **Faithfulness**: Answer consistency with retrieved context
- **Answer Relevancy**: How relevant the answer is to the question
- **Context Precision**: Relevance of retrieved chunks to query
- **Context Recall**: Coverage of necessary information

In [1]:
# Install RAGAS if not already installed
!pip install ragas datasets pandas

  Using cached diskcache-5.6.3-py3-none-any.whl.metadata (20 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 3.9 MB/s  0:00:00 eta 0:00:01
Using cached diskcache-5.6.3-py3-none-any.whl (45 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 8.5 MB/s  0:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.2/603.2 kB 17.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 640.4/640.4 kB 14.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 10.4 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 13.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 kB 18.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 8.3 MB/s  0:00:00m eta 0:00:01
   ━━━━━━━

In [2]:
import boto3
import json
import pandas as pd
import numpy as np
from typing import List, Dict
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall
)
import time

/Users/emota/miniconda3/envs/oreilly/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/jn/py_6v4j90519z5d9j8hh442r0000gn/T/ipykernel_86496/2668963657.py:8: DeprecationWarning: Importing faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import faithfulness
  from ragas.metrics import (
/var/folders/jn/py_6v4j90519z5d9j8hh442r0000gn/T/ipykernel_86496/2668963657.py:8: DeprecationWarning: Importing answer_relevancy from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import answer_relevancy
  from ragas.metrics import (
/var/folders/jn/py_6v4j90519z5d9j8hh442r0000gn/T/ipykernel_86496/26

In [3]:
# Initialize Bedrock clients
bedrock_runtime = boto3.client('bedrock-runtime')
s3 = boto3.client('s3')

# Configuration
RAGAS_BUCKET = f"ragas-evaluation-{int(time.time())}"
EMBEDDING_MODEL = "amazon.titan-embed-text-v1"
GENERATION_MODEL = "amazon.nova-pro-v1:0"

In [4]:
# Create S3 bucket for evaluation data
s3.create_bucket(Bucket=RAGAS_BUCKET)
print(f"Created RAGAS evaluation bucket: {RAGAS_BUCKET}")

Created RAGAS evaluation bucket: ragas-evaluation-1767770912


In [5]:
# Sample knowledge base for evaluation
knowledge_base = [
    {
        "id": "lambda_pricing",
        "content": "AWS Lambda pricing is based on requests and compute time. You pay $0.20 per 1M requests and $0.0000166667 per GB-second. The first 1M requests per month are free."
    },
    {
        "id": "lambda_memory",
        "content": "Lambda memory can be configured from 128 MB to 10,240 MB in 1 MB increments. CPU power scales linearly with memory allocation."
    },
    {
        "id": "lambda_timeout",
        "content": "Lambda functions have a maximum execution time of 15 minutes (900 seconds). The default timeout is 3 seconds."
    },
    {
        "id": "lambda_coldstart",
        "content": "Cold starts occur when Lambda initializes a new execution environment. This adds latency. Use provisioned concurrency to eliminate cold starts."
    },
    {
        "id": "lambda_vpc",
        "content": "Lambda functions can access VPC resources like RDS databases. VPC configuration adds cold start latency. Use VPC endpoints for AWS services."
    }
]

print(f"Knowledge base loaded with {len(knowledge_base)} documents")

Knowledge base loaded with 5 documents


In [6]:
def get_embedding(text: str) -> List[float]:
    """Get embedding using Titan model"""
    response = bedrock_runtime.invoke_model(
        modelId=EMBEDDING_MODEL,
        body=json.dumps({"inputText": text})
    )
    return json.loads(response['body'].read())['embedding']

def cosine_similarity(a: List[float], b: List[float]) -> float:
    """Calculate cosine similarity"""
    a_np = np.array(a)
    b_np = np.array(b)
    return np.dot(a_np, b_np) / (np.linalg.norm(a_np) * np.linalg.norm(b_np))

# Create embeddings for knowledge base
print("Creating embeddings...")
for doc in knowledge_base:
    doc["embedding"] = get_embedding(doc["content"])
    time.sleep(0.1)
print("Embeddings created")

Creating embeddings...
Embeddings created


In [7]:
def retrieve_context(query: str, top_k: int = 3) -> List[str]:
    """Retrieve relevant context for query"""
    query_embedding = get_embedding(query)
    
    similarities = []
    for doc in knowledge_base:
        similarity = cosine_similarity(query_embedding, doc["embedding"])
        similarities.append((doc["content"], similarity))
    
    # Sort by similarity and return top_k contexts
    similarities.sort(key=lambda x: x[1], reverse=True)
    return [content for content, _ in similarities[:top_k]]

def generate_answer(query: str, contexts: List[str]) -> str:
    """Generate answer using Nova Pro"""
    context_text = "\n\n".join(contexts)
    
    prompt = f"""Based on the following context, answer the question accurately.

Context:
{context_text}

Question: {query}

Answer:"""
    
    response = bedrock_runtime.invoke_model(
        modelId=GENERATION_MODEL,
        body=json.dumps({
            "messages": [{
                "role": "user",
                "content": [{"text": prompt}]
            }],
            "inferenceConfig": {
                "maxTokens": 300,
                "temperature": 0.1
            }
        })
    )
    
    result = json.loads(response['body'].read())
    return result['output']['message']['content'][0]['text']

def bedrock_rag_pipeline(query: str) -> Dict:
    """Complete Bedrock RAG pipeline"""
    contexts = retrieve_context(query, top_k=3)
    answer = generate_answer(query, contexts)
    
    return {
        "question": query,
        "contexts": contexts,
        "answer": answer
    }

In [8]:
# Create evaluation dataset
test_questions = [
    "How much does AWS Lambda cost?",
    "What is the maximum memory for Lambda functions?",
    "What is the maximum execution time for Lambda?",
    "How can I reduce Lambda cold starts?",
    "Can Lambda functions access VPC resources?",
    "What is the default timeout for Lambda functions?",
    "How does Lambda pricing work?",
    "What causes Lambda cold starts?"
]

# Ground truth answers for reference (optional for RAGAS)
ground_truth = [
    "Lambda costs $0.20 per 1M requests and $0.0000166667 per GB-second, with 1M free requests monthly.",
    "Lambda memory can be configured up to 10,240 MB in 1 MB increments.",
    "Lambda functions have a maximum execution time of 15 minutes (900 seconds).",
    "Use provisioned concurrency to eliminate cold starts for Lambda functions.",
    "Yes, Lambda functions can access VPC resources like RDS databases, but this adds cold start latency.",
    "The default timeout for Lambda functions is 3 seconds.",
    "Lambda pricing is based on the number of requests and compute time (GB-seconds).",
    "Cold starts occur when Lambda initializes a new execution environment after being idle."
]

print(f"Created evaluation dataset with {len(test_questions)} questions")

Created evaluation dataset with 8 questions


In [9]:
# Generate RAG responses for evaluation
print("Generating RAG responses...")
evaluation_data = []

for i, question in enumerate(test_questions):
    print(f"Processing question {i+1}/{len(test_questions)}: {question[:50]}...")
    
    # Get RAG response
    rag_result = bedrock_rag_pipeline(question)
    
    evaluation_data.append({
        "question": question,
        "contexts": rag_result["contexts"],
        "answer": rag_result["answer"],
        "ground_truth": ground_truth[i] if i < len(ground_truth) else ""
    })
    
    time.sleep(0.5)  # Rate limiting

print("RAG responses generated")

Generating RAG responses...
Processing question 1/8: How much does AWS Lambda cost?...
Processing question 2/8: What is the maximum memory for Lambda functions?...
Processing question 3/8: What is the maximum execution time for Lambda?...
Processing question 4/8: How can I reduce Lambda cold starts?...
Processing question 5/8: Can Lambda functions access VPC resources?...
Processing question 6/8: What is the default timeout for Lambda functions?...
Processing question 7/8: How does Lambda pricing work?...
Processing question 8/8: What causes Lambda cold starts?...
RAG responses generated


In [10]:
# Convert to RAGAS dataset format
ragas_dataset = Dataset.from_dict({
    "question": [item["question"] for item in evaluation_data],
    "contexts": [item["contexts"] for item in evaluation_data],
    "answer": [item["answer"] for item in evaluation_data],
    "ground_truth": [item["ground_truth"] for item in evaluation_data]
})

print(f"Created RAGAS dataset with {len(ragas_dataset)} samples")
print("\nSample data:")
print(f"Question: {ragas_dataset[0]['question']}")
print(f"Answer: {ragas_dataset[0]['answer'][:100]}...")
print(f"Contexts: {len(ragas_dataset[0]['contexts'])} retrieved")

Created RAGAS dataset with 8 samples

Sample data:
Question: How much does AWS Lambda cost?
Answer: The cost of using AWS Lambda depends on two main factors: the number of requests and the compute tim...
Contexts: 3 retrieved


In [12]:
# Configure RAGAS to use Bedrock models
import os
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_aws import BedrockLLM, BedrockEmbeddings

# Note: This is a simplified approach. In practice, you might need to configure
# RAGAS to work with Bedrock models directly or use OpenAI for evaluation

# For this demo, we'll use the default RAGAS configuration
# In production, configure with your preferred evaluation models

print("RAGAS configuration ready")

RAGAS configuration ready


In [13]:
# Run RAGAS evaluation
print("Running RAGAS evaluation...")
print("Note: This requires OpenAI API key or custom model configuration")

# Metrics to evaluate
metrics = [
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall
]

try:
    # Run evaluation
    result = evaluate(
        dataset=ragas_dataset,
        metrics=metrics
    )
    
    print("\nRAGAS Evaluation Results:")
    print("=" * 40)
    
    for metric_name, score in result.items():
        print(f"{metric_name}: {score:.4f}")
    
    # Convert to DataFrame for detailed analysis
    results_df = result.to_pandas()
    print(f"\nDetailed results saved to DataFrame with {len(results_df)} rows")
    
except Exception as e:
    print(f"RAGAS evaluation error: {e}")
    print("This typically requires OpenAI API configuration or custom model setup")
    
    # Fallback: Manual evaluation demonstration
    print("\nDemonstrating manual evaluation approach...")
    
    # Simple manual evaluation metrics
    manual_scores = {
        "faithfulness": [],
        "answer_relevancy": [],
        "context_precision": [],
        "context_recall": []
    }
    
    for item in evaluation_data:
        # Simple heuristic scoring (0-1 scale)
        answer_length = len(item["answer"].split())
        context_count = len(item["contexts"])
        
        # Mock scores based on simple heuristics
        manual_scores["faithfulness"].append(min(1.0, answer_length / 50))
        manual_scores["answer_relevancy"].append(min(1.0, answer_length / 30))
        manual_scores["context_precision"].append(min(1.0, context_count / 3))
        manual_scores["context_recall"].append(min(1.0, context_count / 3))
    
    print("\nManual Evaluation Results (Heuristic):")
    print("=" * 40)
    for metric, scores in manual_scores.items():
        avg_score = sum(scores) / len(scores)
        print(f"{metric}: {avg_score:.4f}")

Running RAGAS evaluation...
Note: This requires OpenAI API key or custom model configuration
RAGAS evaluation error: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable
This typically requires OpenAI API configuration or custom model setup

Demonstrating manual evaluation approach...

Manual Evaluation Results (Heuristic):
faithfulness: 0.6850
answer_relevancy: 0.7625
context_precision: 1.0000
context_recall: 1.0000


In [14]:
# Analyze results by question
print("\nPER-QUESTION ANALYSIS")
print("=" * 50)

for i, item in enumerate(evaluation_data):
    print(f"\nQuestion {i+1}: {item['question']}")
    print(f"Answer: {item['answer'][:100]}...")
    print(f"Contexts retrieved: {len(item['contexts'])}")
    print(f"Answer length: {len(item['answer'].split())} words")
    
    # Show first context for reference
    if item['contexts']:
        print(f"Top context: {item['contexts'][0][:80]}...")


PER-QUESTION ANALYSIS

Question 1: How much does AWS Lambda cost?
Answer: The cost of using AWS Lambda depends on two main factors: the number of requests and the compute tim...
Contexts retrieved: 3
Answer length: 159 words
Top context: AWS Lambda pricing is based on requests and compute time. You pay $0.20 per 1M r...

Question 2: What is the maximum memory for Lambda functions?
Answer: The maximum memory for Lambda functions is 10,240 MB (10 GB)....
Contexts retrieved: 3
Answer length: 11 words
Top context: Lambda memory can be configured from 128 MB to 10,240 MB in 1 MB increments. CPU...

Question 3: What is the maximum execution time for Lambda?
Answer: The maximum execution time for AWS Lambda functions is 15 minutes (900 seconds)....
Contexts retrieved: 3
Answer length: 13 words
Top context: Lambda functions have a maximum execution time of 15 minutes (900 seconds). The ...

Question 4: How can I reduce Lambda cold starts?
Answer: To reduce Lambda cold starts, you can take the

In [ ]:
# Compare different RAG configurations (simulation)
print("\nRAG CONFIGURATION COMPARISON")
print("=" * 50)

# Simulate different configurations
configurations = {
    "Basic RAG (top-3)": {"faithfulness": 0.85, "answer_relevancy": 0.82, "context_precision": 0.78, "context_recall": 0.80},
    "Hybrid Search": {"faithfulness": 0.88, "answer_relevancy": 0.85, "context_precision": 0.82, "context_recall": 0.84},
    "Re-ranking Pipeline": {"faithfulness": 0.91, "answer_relevancy": 0.89, "context_precision": 0.87, "context_recall": 0.85},
    "Multi-Collection": {"faithfulness": 0.87, "answer_relevancy": 0.86, "context_precision": 0.84, "context_recall": 0.83}
}

# Create comparison DataFrame
comparison_df = pd.DataFrame(configurations).T
print(comparison_df)

# Calculate overall scores
comparison_df['Overall'] = comparison_df.mean(axis=1)
print("\nOverall Scores:")
print(comparison_df['Overall'].sort_values(ascending=False))


RAG CONFIGURATION COMPARISON
                     faithfulness  answer_relevancy  context_precision  \
Basic RAG (top-3)            0.85              0.82               0.78   
Hybrid Search                0.88              0.85               0.82   
Re-ranking Pipeline          0.91              0.89               0.87   
Multi-Collection             0.87              0.86               0.84   

                     context_recall  
Basic RAG (top-3)              0.80  
Hybrid Search                  0.84  
Re-ranking Pipeline            0.85  
Multi-Collection               0.83  

Overall Scores:
Re-ranking Pipeline    0.8800
Multi-Collection       0.8500
Hybrid Search          0.8475
Basic RAG (top-3)      0.8125
Name: Overall, dtype: float64


## RAGAS Metrics Explained

### **Faithfulness** (0-1 scale)
- Measures if the generated answer is factually consistent with retrieved context
- Higher scores indicate answers grounded in provided evidence
- **Good**: Answer claims can be verified from context
- **Bad**: Answer contains information not in context

### **Answer Relevancy** (0-1 scale)
- Evaluates how relevant the answer is to the original question
- Higher scores indicate direct, on-topic responses
- **Good**: Answer directly addresses the question
- **Bad**: Answer is off-topic or too generic

### **Context Precision** (0-1 scale)
- Measures the proportion of relevant chunks in retrieved contexts
- Higher scores indicate better retrieval quality
- **Good**: Most retrieved contexts are relevant to query
- **Bad**: Many irrelevant contexts retrieved

### **Context Recall** (0-1 scale)
- Determines if all necessary information was retrieved
- Higher scores indicate comprehensive information retrieval
- **Good**: All information needed to answer is retrieved
- **Bad**: Missing key information in retrieved contexts

## Interpretation Guidelines:
- **0.8+**: Excellent performance
- **0.6-0.8**: Good performance
- **0.4-0.6**: Needs improvement
- **<0.4**: Poor performance, requires optimization

## Using RAGAS for RAG Optimization:
1. **Low Faithfulness**: Improve context relevance or generation prompts
2. **Low Answer Relevancy**: Refine query understanding or response generation
3. **Low Context Precision**: Improve retrieval algorithms or embeddings
4. **Low Context Recall**: Increase retrieval count or improve chunking strategy

In [16]:
# Save evaluation results
evaluation_summary = {
    "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
    "total_questions": len(test_questions),
    "knowledge_base_size": len(knowledge_base),
    "embedding_model": EMBEDDING_MODEL,
    "generation_model": GENERATION_MODEL,
    "evaluation_data": evaluation_data
}

# Store in S3
s3.put_object(
    Bucket=RAGAS_BUCKET,
    Key="evaluation_results.json",
    Body=json.dumps(evaluation_summary, indent=2)
)

print(f"\nEvaluation complete!")
print(f"Results saved to S3 bucket: {RAGAS_BUCKET}")
print(f"\nRAGAS provides objective metrics to:")
print(f"- Compare different RAG configurations")
print(f"- Monitor production RAG performance")
print(f"- Identify areas for improvement")
print(f"- A/B test RAG optimizations")


Evaluation complete!
Results saved to S3 bucket: ragas-evaluation-1767770912

RAGAS provides objective metrics to:
- Compare different RAG configurations
- Monitor production RAG performance
- Identify areas for improvement
- A/B test RAG optimizations
